In [7]:
from csv import DictReader
from tabulate import tabulate
from random import randrange

6


### Helper Functions

In [93]:
class Villager:
    def __init__(self, id, product, data):
        """ 
        Creates a villager based on units consumed per person per week.
        Inventory will be initialized with negative values as initially a villager will be in deficit of those values
        """
        self.id = id
        self.product = product
        self.inventory = {}
        for row in data:
            self.inventory[row['product']] = -1 * float(row['units_consumed_pppw'])

    def __str__(self):
        return str(self.id)

    def add_product(self, product, units):
        """ updates villager's inventory by giving units of a product to them """
        self.inventory[product] += units

    def remove_product(self, product, units):
        """ updates villager's inventory by taking units of a product from them """
        self.inventory[product] -= units
        
    def set_product(self, product, units):
        """ updates villager's inventory by setting product to given units """
        self.inventory[product] = units

    def has_product(self, product, min_units):
        """ checks if the villager has sufficient units of product """
        return self.inventory[product] >= min_units

    def can_sell(self):
        """ returns list of product surplus in inventory that villager wants to sell """
        surplus = []
        for product in self.inventory:
            if self.inventory[product] > 0:
                surplus.append(product)
        return surplus

    def can_buy(self):
        """ returns list of product deficit in inventory that villager wants to buy """
        deficit = []
        for product in self.inventory:
            if self.inventory[product] < 0:
                deficit.append(product)
        return deficit

    def has_product_to_sell(self):
        return len(self.can_sell()) > 0

    def has_product_to_buy(self):
        return len(self.can_buy()) > 0


def load_csv(file):
    """ loads csv file to dict """
    with open(file, 'r', encoding='utf-8') as f:
        dict_reader = DictReader(f)
        return list(dict_reader)

def get_producer_distribution(population, data):
    """ defines number of producers for a product based on population and producers_percent """
    producer_distribution = {}
    for row in data:
        producer_distribution[row['product']] = round((population * float(row['producers_percent']))/100)
    return producer_distribution

def get_units_produced_per_hour(data):
    """ returns units_produced_per_hour from data """
    units_produced_per_hour = {}
    for row in data:
        units_produced_per_hour[row['product']] = float(row['units_produced_per_hour'])
    return units_produced_per_hour

def create_village(producer_distribution, units_produced_per_hour, work_hours):
    """ 
    Creates a village by adding villagers based on producer_distribution.
    Initially each villager will get units equivalent of production_rate times work_hours of their product
    """
    village = []
    id = 0
    for product in producer_distribution:
        for producers in range(producer_distribution[product]):
            villager = Villager(id, product, data)
            villager.add_product(product, units_produced_per_hour[product] * work_hours)
            village.append(villager)
            id += 1
    return village

def print_village(village):
    """ Prints inventory of all the villagers """
    village_data = []
    for villager in village:
        formatted_villager = vars(villager) | villager.inventory
        formatted_villager.pop('inventory', None)
        village_data.append(formatted_villager)

    print(tabulate(village_data, headers='keys', tablefmt='fancy_grid'))

def get_exchange_rate(units_produced_per_hour):
    """ creates two dimensional table of exchange rates between the produces """
    exchange_rate = {}
    for product1 in units_produced_per_hour:
        product1_exchange_rates = {}
        for product2 in units_produced_per_hour:
            product1_exchange_rates[product2] = units_produced_per_hour[product2]/units_produced_per_hour[product1]
        exchange_rate[product1] = product1_exchange_rates
    return exchange_rate

def print_exchange_rate(exchange_rate):
    """ Prints exchange rate table for 1 unit of product on y-axis get number of units of product on x-axis """
    exchange_rate_table = []
    for product in exchange_rate:
        exchange_rate_table.append({'↓ 1 unit \\ gets →':product} | exchange_rate[product])  
    print(tabulate(exchange_rate_table, headers='keys', tablefmt='fancy_grid'))

class Market:
    def __init__(self, village, exchange_rate):
        """ Initializes the market by identifying villagers willing to sell or buy products """
        self.market_is_open = True
        self.buyers = {}
        self.sellers = {}
        self.transactions = []
        self.exchange_rate = exchange_rate
        
        buyer_products = set()
        seller_products = set()

        for villager in village:
            if villager.has_product_to_buy():
                for product in villager.can_buy():
                    self.add_buyer(product, villager)
                    buyer_products.add(product)
                
            if villager.has_product_to_sell():
                for product in villager.can_sell():
                    self.add_seller(product, villager)
                    seller_products.add(product)

        self.products = buyer_products.intersection(seller_products)

    @staticmethod
    def add_villager(trader_dict, product, villager):
        """ Adds a villager to the product list in trader_dict, if product is not present the creates a new list """
        if product in trader_dict:
            trader_dict[product].append(villager)
        else:
            trader_dict[product] = [villager]

    def add_buyer(self, product, villager):
        Market.add_villager(self.buyers, product, villager)

    def add_seller(self, product, villager):
        Market.add_villager(self.sellers, product, villager)

    @staticmethod
    def exchange(trader1, product1, product1_units, trader2, product2, product2_units):
        """ Performs exchange of product of given units between traders """
        trader1.remove_product(product1, product1_units)
        trader2.remove_product(product2, product2_units)
        trader1.add_product(product2, product2_units)
        trader2.add_product(product1, product1_units)

    @staticmethod
    def trade(trader1, product1, product1_units, trader2, product2, exchange_rate):
        """
        Perform trade of product1, product2 between trader1, trader2 for provided units and exchange rate.
        If both the villagers have sufficient units then complete trade is executed else partial trade is executed. 
        """
        product2_units = product1_units * exchange_rate[product1][product2]
        if trader1.has_product(product1, product1_units) and trader2.has_product(product2, product2_units):
            exchange(trader1, product1, product1_units, trader2, product2, product2_units)
        else:
            trader1_capacity = trader1.inventory[product1]
            trader2_capacity = trader2.inventory[product2] * exchange_rate[product1][product2]
            product1_units = min(trader1_capacity, trader2_capacity)
            product2_units = product1_units * exchange_rate[product1][product2]
            exchange(trader1, product1, product1_units, trader2, product2, product2_units)
        
        return {
            'trader1': trader1, 'product1': product1, 'product1_units': product1_units, 
            'trader2': trader2, 'product2': product2, 'product2_units': product2_units
            }

    def execute(self):
        """ 
        Simulates the market by picking random product and then performs trading between random buyer and
        random seller. When a villager has no deficit or no surplus then the villager is removed from the buyers
        or sellers list for the product. When a product has no buyers or sellers, then the product is removed
        from the products list. Market remains open until products list is not empty.
        """
        while self.market_is_open:
            product = list(self.products)[randrange(len(self.products))]
            trader1 = self.buyers[product][randrange(len(self.buyers[product]))]
            if trader1.has_product_to_sell():
                product_to_exchange = trader1.can_sell()[randrange(len(trader1.can_sell()))]
                trader2 = self.sellers[product][randrange(len(self.sellers[product]))]
                units = -1 * trader2.inventory[product]
                self.transactions.append(trade(trader1, product, units, trader2, product_to_exchange, exchange_rate))

                if not product in trader1.can_buy():
                    self.buyers[product].remove(trader1)
                if not product_to_exchange in trader1.can_sell():
                    self.sellers[product].remove(trader1)
                if not product in trader2.can_sell():
                    self.sellers[product].remove(trader2)
                if not product_to_exchange in trader2.can_buy():
                    self.buyers[product].remove(trader2)

            else:
                self.buyers[product].remove(trader1)
            
            if len(sellers[product] == 0):
                self.sellers.pop(product, None)
                self.products.remove(product)

            if len(buyers[product] == 0):
                self.buyers.pop(product, None)
                self.products.remove(product)

            if len(sellers[product_to_exchange] == 0):
                self.sellers.pop(product_to_exchange, None)
                self.products.remove(product_to_exchange)

            if len(buyers[product_to_exchange] == 0):
                self.buyers.pop(product_to_exchange, None)
                self.products.remove(product_to_exchange)

            self.market_is_open = len(self.products) > 0

            



### Initialise

In [94]:
data = load_csv('data.csv')
population = 100
work_hours = 40

producer_distribution = get_producer_distribution(population, data)
units_produced_per_hour = get_units_produced_per_hour(data)
exchange_rate = get_exchange_rate(units_produced_per_hour)
village = create_village(producer_distribution, units_produced_per_hour, work_hours)

In [95]:
print_exchange_rate(exchange_rate)

╒═══════════════════════╤═══════════╤════════╤═══════════╤══════════════╤══════════╤═══════════╤═══════════╤═══════════╤════════╤═══════════╤═══════════╤══════════╤═══════════╤══════════════════╤═══════════╤═══════════╤═════════════════════════╤═══════════╤═══════════╤═══════════╕
│ ↓ 1 unit \ gets →     │     wheat │   rice │      maze │   vegetables │   fruits │      fish │   chicken │      meat │   eggs │      milk │     metal │     wood │     cloth │   transportation │   tourism │    health │   constructions_repairs │   council │       law │    police │
╞═══════════════════════╪═══════════╪════════╪═══════════╪══════════════╪══════════╪═══════════╪═══════════╪═══════════╪════════╪═══════════╪═══════════╪══════════╪═══════════╪══════════════════╪═══════════╪═══════════╪═════════════════════════╪═══════════╪═══════════╪═══════════╡
│ wheat                 │  1        │    1.5 │  0.5      │     0.25     │ 0.25     │  1        │  0.5      │ 0.1       │    1.5 │  0.5      │  1.25     │ 

In [96]:
print_village(village)

╒══════╤═══════════════════════╤═════════╤════════╤════════╤══════════════╤══════════╤════════╤═══════════╤════════╤════════╤════════╤═════════╤════════╤═════════╤══════════════════╤═══════════╤══════════╤═════════════════════════╤═══════════╤═══════╤══════════╕
│   id │ product               │   wheat │   rice │   maze │   vegetables │   fruits │   fish │   chicken │   meat │   eggs │   milk │   metal │   wood │   cloth │   transportation │   tourism │   health │   constructions_repairs │   council │   law │   police │
╞══════╪═══════════════════════╪═════════╪════════╪════════╪══════════════╪══════════╪════════╪═══════════╪════════╪════════╪════════╪═════════╪════════╪═════════╪══════════════════╪═══════════╪══════════╪═════════════════════════╪═══════════╪═══════╪══════════╡
│    0 │ wheat                 │      76 │     -6 │     -2 │           -3 │       -2 │   -3.2 │        -2 │  -0.32 │     -6 │   -1.6 │      -2 │   -1.6 │    -0.2 │             -2.4 │      -0.8 │     -0.4 │      

### Execute

In [97]:
market = Market(village, exchange_rate)
market.execute()

police
[<__main__.Villager object at 0x7f263bcd0340>, <__main__.Villager object at 0x7f263bcd0d90>, <__main__.Villager object at 0x7f263bcd1150>]
38


ValueError: list.remove(x): x not in list

In [ ]:
# Scratchpad
market.transactions

NameError: name 'market' is not defined